In [1]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import akshare as ak
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from database.downloader.downloader_base import DownloaderBase
import database.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

!python --version

Python 3.8.10


In [2]:
import tensorflow as tf

# 只使用CPU进行训练
tf.config.set_visible_devices([], 'GPU')

# 打印Tensorflow版本
print(f"Tensorflow Version: {tf.__version__}")

# 检查是否有可用的GPU设备
if tf.test.is_built_with_cuda():
    print("TensorFlow GPU version is installed")
else:
    print("TensorFlow CPU version is installed")

# 检查TensorFlow是否能够访问GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU devices available:", gpus)
else:
    print("No GPU devices found. Running on CPU.")

# !nvidia-smi

2024-04-18 11:03:13.724267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 11:03:14.585595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tensorflow Version: 2.13.1
TensorFlow GPU version is installed
GPU devices available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer, StandardScaler

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# 绘图相关函数
def plot_series_dist(series):
    data = series
    plt.figure(figsize=(5,5))
    # 使用matplotlib画直方图
    plt.hist(data, bins=60, edgecolor='k', alpha=0.7)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.show()

def plot_metrics(history):
  metrics = ['loss', 'mean_absolute_error', 'mean_squared_error']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.ylim([0, plt.ylim()[1]])
    plt.legend()

def plot_cm(true_labels, pred_labels):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="g", cmap='Blues')
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

In [13]:
class PreProcessing:
    def __init__(self, db_downloader:DownloaderBase) -> None:
        self.db_downloader = db_downloader

    def _build_cls_label(self, stock_dataframe):
        def calculate_atr(df, period=14):
            df['high-low'] = df['high'] - df['low']
            df['high-close_prev'] = abs(df['high'] - df['close'].shift(1))
            df['low-close_prev'] = abs(df['low'] - df['close'].shift(1))
            df['tr'] = df[['high-low', 'high-close_prev', 'low-close_prev']].max(axis=1)
            atr = df['tr'].rolling(window=period, min_periods=1).mean()
            return atr

        # 初始化标签参数
        N = 30 # 预测时间周期
        ATR_period = 14 # ATR计算周期
        ATR_take_profit_factor = 3 # 止盈参数
        ATR_stop_loss_factor = 2 # 止损参数
        # 开始构建标签
        df = stock_dataframe.copy()
        df['atr'] = calculate_atr(df, period=ATR_period) # 计算每一天的ATR
        df['label'] = 0  # 初始化标签列
        # 轮询判断先止盈还是先止损
        for index in range(len(df)-1):
            buy_open_price = df.at[index + 1, 'open'] # 第二天的开盘价作为买入价
            for day in range(2, min(N+2, len(df) - index)):
                if index + day >= len(df):
                    break
                future_day_close = df.at[index+day, 'close'] # 第三天的收盘价
                future_day_atr = df.at[index+day, 'atr']
                # 动态计算止盈止损价格
                take_profit_price = buy_open_price + ATR_take_profit_factor * future_day_atr
                stop_loss_price = buy_open_price - ATR_stop_loss_factor * future_day_atr
                # 检查当天价格是否触发止盈或止损条件
                if future_day_close >= take_profit_price:
                    df.at[index, 'label'] = 1  # 止盈被触发
                    break  # 退出内循环
                elif future_day_close <= stop_loss_price:
                    df.at[index, 'label'] = 2  # 止损被触发
                    break  # 退出内循环
        return df[['datetime', 'label']]

    def _process_one_stock(self, stock_code, start_date, end_date):
        stock_base = self.db_downloader._download_stock_base_info(stock_code) # 获取基础代码
        stock_individual = self.db_downloader._download_stock_individual_info(stock_code) # 获取profile信息
        stock_history = self.db_downloader._download_stock_history_info(stock_code, start_date, end_date) # 获取历史行情
        stock_indicator = self.db_downloader._download_stock_indicator_info(stock_code, start_date, end_date) # 获取指标数据
        stock_factor_date = self.db_downloader._download_stock_factor_date_info() # 获取日期特征
        stock_factor_qlib = self.db_downloader._download_stock_factor_qlib_info(stock_code, start_date, end_date) # 获取量价特征
        stock_label = self._build_cls_label(stock_history, ) # 构建Label
        stock_df = stock_base.merge(stock_individual, on=['stock_code']).merge(stock_history, on=['stock_code']).merge(stock_indicator, on=['stock_code', 'datetime']).merge(stock_label, on=['datetime']).merge(stock_factor_date, on=['datetime']).merge(stock_factor_qlib, on=['stock_code', 'datetime']) # 整合数据
        stock_df = stock_base \
            .merge(stock_individual, on=['stock_code', 'stock_name']) \
            .merge(stock_history, on=['stock_code']) \
            .merge(stock_indicator, on=['stock_code', 'datetime']) \
            .merge(stock_label, on=['datetime']) \
            .merge(stock_factor_date, on=['datetime']) \
            .merge(stock_factor_qlib, on=['stock_code', 'datetime']) # 整合数据
        stock_df = stock_df.dropna()
        return stock_df
    
    def _process_all_stock(self, code_type, start_date, end_date):
        # stock_code_list = list(ak.stock_info_a_code_name()['code'].unique()) # 获取A股所有股票列表
        stock_code_list = list(ak.index_stock_cons(code_type)['品种代码'].unique()) # 获取沪深300的股票代码列表
        stock_df_list = []
        for stock_code in tqdm(stock_code_list, desc=f'Process: {code_type} ...'):
            stock_df = self._process_one_stock(stock_code, start_date, end_date)
            if not stock_df.empty:
                stock_df_list.append(stock_df)
        return pd.concat(stock_df_list)

In [14]:
# 数据库初始化
db_conn = sqlite3.connect('../database/hh_quant.db')
db_downloader = DownloaderBase(db_conn, db_config)
proprocessor = PreProcessing(db_downloader=db_downloader)
df = proprocessor._process_all_stock('000016', '20000101', '20050101')

Process: 000016 ...: 100%|██████████| 50/50 [00:06<00:00,  7.53it/s]


In [16]:
# df['label'].value_counts()

In [17]:
def get_rolling_data_period(backtest_start_date, backtest_duration=5, train_period=6, val_period=0.5, test_period=0.5):
    """
    Args:
        backtest_start_date (_type_): _description_
        backtest_duration (int, optional): _description_. Defaults to 5.
        train_period (int, optional): _description_. Defaults to 6.
        val_period (float, optional): _description_. Defaults to 0.5.
        test_period (float, optional): _description_. Defaults to 0.5.
    Returns:
        result: _description_
    """
    backtest_start_date = datetime.strptime(backtest_start_date, '%Y%m%d')
    backtest_end_date = backtest_start_date + relativedelta(years=backtest_duration) # 回测5年数据
    train_period = relativedelta(years=train_period) # 使用6年的训练数据
    val_period = relativedelta(months=(12 * val_period)) # 使用半年的验证数据
    test_period = relativedelta(months=(12 * test_period)) # 使用半年的测试数据(半年模型一更新)

    result = []
    rolling_flag = True
    bench_date = backtest_start_date
    while rolling_flag:
        if bench_date < backtest_end_date:
            test_start, test_end = bench_date, (bench_date + test_period - relativedelta(days=1))
            val_start, val_end = (test_start - relativedelta(days=1) - val_period), (test_start - relativedelta(days=1))
            train_start, train_end =(val_start - relativedelta(days=1) - train_period), (val_start - relativedelta(days=1))
            result.append({
                "train": [train_start.strftime("%Y%m%d"), train_end.strftime("%Y%m%d")],
                "val": [val_start.strftime("%Y%m%d"), val_end.strftime("%Y%m%d")],
                "test": [test_start.strftime("%Y%m%d"), test_end.strftime("%Y%m%d")]
            })
            bench_date += test_period
        else:
            rolling_flag = False 
    return result

def extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date, test_start_date, test_end_date):
    train_start_date = pd.to_datetime(train_start_date)
    train_end_date = pd.to_datetime(train_end_date)
    val_start_date = pd.to_datetime(val_start_date)
    val_end_date = pd.to_datetime(val_end_date)
    test_start_date = pd.to_datetime(test_start_date)
    test_end_date = pd.to_datetime(test_end_date)

    train_data = df[(pd.to_datetime(df['datetime']) >= train_start_date) & (pd.to_datetime(df['datetime']) <= train_end_date)]
    val_data = df[(pd.to_datetime(df['datetime']) >= val_start_date) & (pd.to_datetime(df['datetime']) <= val_end_date)]
    test_data = df[(pd.to_datetime(df['datetime']) >= test_start_date) & (pd.to_datetime(df['datetime']) <= test_end_date)]

    print(f"train_data_size: {train_data.shape}")
    print(f"validation_data_size: {val_data.shape}")
    print(f"test_data_size: {test_data.shape}")
    return train_data, val_data, test_data

def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=True, batch_size=32):
    features = dataframe[feature_cols]
    labels = dataframe[label_cols]
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(features), 10000))
    ds = ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [18]:
# 相关配置
rolling_flag = False
benchmark = '000016'
feature_config = {
    "target_features": ['label'],
    "numeric_features": ['turnover_rate', 'pe_ttm', 'ps_ttm', 'pcf_ncf_ttm', 'pb_mrq', 'KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60'],
    "integer_categorical_features": ['month'],
    "string_categorical_features": ['industry', 'season'],
}
batch_size = 1024

# 是否开启滚动训练&回测
if rolling_flag:
    print("开启滚动回测...")
    backtest_period = get_rolling_data_period(
        backtest_start_date='20200101', # 回测开始日期
        backtest_duration=4, # 一共回测多久的数据（单位：年）
        train_period=6, # 使用过去多久的时间进行训练（单位：年）
        val_period=1, # 验证数据周期（单位：年）
        test_period=1, # 测试数据周期（单位：年）
    )
else:
    print("关闭滚动回测...")
    backtest_period = [
        {
            'train': ['20130101', '20181231'],
            'val': ['20190101', '20191231'],
            'test': ['20200101', '20231231']
        }
    ]

backtest_period

关闭滚动回测...


[{'train': ['20130101', '20181231'],
  'val': ['20190101', '20191231'],
  'test': ['20200101', '20231231']}]

In [19]:
# 训练执行
for date_period_params in tqdm(backtest_period, desc='Training_Period...'):
    print(date_period_params)
    train_start_date, train_end_date = date_period_params['train']
    val_start_date, val_end_date = date_period_params['val']
    test_start_date, test_end_date = date_period_params['test']
    # 获取全区间数据
    print("开始加载原始数据...")
    df = proprocessor._process_all_stock(code_type=benchmark, start_date=train_start_date, end_date=test_end_date)
    # 抽取训练验证数据
    print("开始拆分训练、验证、测试集合...")
    train_data, val_data, test_data = extract_train_val_data(df, *[train_start_date, train_end_date, val_start_date, val_end_date, test_start_date, test_end_date])
    # 计算类别权重
    print("开始计算类别权重...")
    value_count = train_data['label'].value_counts()
    total_count = train_data['label'].count()
    class_weight = ((1 / value_count) * (total_count / 2.0)).to_dict()
    print(class_weight)
    # 从data中抽取相关特征数据
    print("开始抽取特征数据...")
    feature_columns = feature_config.get('numeric_features', []) + feature_config.get('integer_categorical_features', []) + feature_config.get('string_categorical_features', [])
    label_columns = feature_config.get('target_features', [])
    full_feature_columns = feature_columns + label_columns
    train_df, val_df, test_df = train_data[full_feature_columns], val_data[full_feature_columns], test_data[full_feature_columns]
    # 对相关特征进行特征工程
    print("开始特征工程处理...")
    preprocessing_pipeline = Pipeline([
            ('quantile_transformer', QuantileTransformer(output_distribution='uniform', n_quantiles=1000)),
        ])
    numeric_feature_columns = feature_config.get('numeric_features', [])
    train_df[numeric_feature_columns] = preprocessing_pipeline.fit_transform(train_df[numeric_feature_columns])
    val_df[numeric_feature_columns] = preprocessing_pipeline.transform(val_df[numeric_feature_columns])
    test_df[numeric_feature_columns] = preprocessing_pipeline.transform(test_df[numeric_feature_columns])
    # 转换为tensorflow所使用的dataset
    print("开始将DataFrame转换为DataSet...")
    train_ds = df_to_dataset(train_df, feature_columns, label_columns, shuffle=True, batch_size=batch_size)
    val_ds = df_to_dataset(val_df, feature_columns, label_columns, shuffle=False, batch_size=batch_size)
    test_ds = df_to_dataset(test_df, feature_columns, label_columns, shuffle=False, batch_size=batch_size)  
    # 准备模型训练
    print("开始模型初始化 & 训练...")
    # from models.single_task.model_base import QuantModel
    # from models.single_task.model_senet import QuantModel
    from models.single_task.model_moe import QuantModel
    model_config = {
            "seed": 1024,
            "feature_embedding_dims": 4,
            "output_dim": 3,
            "integer_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('integer_categorical_features', [])},
            "string_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('string_categorical_features', [])},
        }
    model = QuantModel(config=model_config)

    initial_learning_rate = 5e-4
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        initial_learning_rate,
        decay_steps=(len(train_data) // batch_size)*10,
        decay_rate=1,
        staircase=False)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    EPOCHS = 500

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        verbose=1,
        patience=10,
        mode='min',
        restore_best_weights=True)

    baseline_history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[early_stopping],
        class_weight=class_weight
    )
    
    # 保存模型
    # print("开始保存模型...")
    # model_save_path = f'./models/saved_model/{benchmark}/model_of_reg_{test_start_date}'
    # model.save(model_save_path)
    # model = tf.keras.models.load_model(model_save_path)

    # 输出回测预测
    # print("开始保存回测预测结果...")
    # model_pred_result = model.predict(test_ds)
    # output_df = test_data[['stock_code', 'industry', 'stock_name', 'datetime']]
    # output_df['label'] = test_data['label']
    # output_df['label_pred'] = model_pred_result
    # if rolling_flag:
    #     output_df.to_pickle(f'../../Offline/backtest/backtest_data/{benchmark}/stock_selection_results_{test_start_date}.pkl') 
    # else:
    #     output_df.to_pickle(f'../../Offline/backtest/backtest_data/test/{benchmark}_stock_selection_results_{test_start_date}.pkl') 

    # # 绘制结果
    # test_true_label = test_data['label']
    # test_pred_label = model_pred_result
    # plot_cm(
    #     true_labels = (np.array(test_true_label) > 0).astype(int), 
    #     pred_labels = (np.array(test_pred_label) > 0).astype(int)
    # )


Training_Period...:   0%|          | 0/1 [00:00<?, ?it/s]

{'train': ['20130101', '20181231'], 'val': ['20190101', '20191231'], 'test': ['20200101', '20231231']}
开始加载原始数据...


Process: 000016 ...: 100%|██████████| 50/50 [00:34<00:00,  1.47it/s]


开始拆分训练、验证、测试集合...
train_data_size: (52468, 207)
validation_data_size: (10114, 207)
test_data_size: (45766, 207)
开始计算类别权重...
{2: 1.13341398081742, 1: 1.3025172533637854, 0: 2.8574229386777037}
开始抽取特征数据...
开始特征工程处理...
开始将DataFrame转换为DataSet...
开始模型初始化 & 训练...
Epoch 1/500


2024-04-18 11:12:18.105559: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1517b027ecf0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-18 11:12:18.105648: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-04-18 11:12:18.192045: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-18 11:12:18.382404: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


52/52 [==============================] - 13s 149ms/step - loss: 2.3922 - sparse_categorical_accuracy: 0.3449 - val_loss: 1.1460 - val_sparse_categorical_accuracy: 0.3487
Epoch 2/500
52/52 [==============================] - 3s 65ms/step - loss: 2.0939 - sparse_categorical_accuracy: 0.3482 - val_loss: 1.1815 - val_sparse_categorical_accuracy: 0.3157
Epoch 3/500
52/52 [==============================] - 3s 66ms/step - loss: 1.9852 - sparse_categorical_accuracy: 0.3514 - val_loss: 1.1644 - val_sparse_categorical_accuracy: 0.3281
Epoch 4/500
52/52 [==============================] - 4s 70ms/step - loss: 1.9077 - sparse_categorical_accuracy: 0.3574 - val_loss: 1.1536 - val_sparse_categorical_accuracy: 0.3311
Epoch 5/500
52/52 [==============================] - 3s 67ms/step - loss: 1.8524 - sparse_categorical_accuracy: 0.3608 - val_loss: 1.1314 - val_sparse_categorical_accuracy: 0.3412
Epoch 6/500
52/52 [==============================] - 3s 64ms/step - loss: 1.8248 - sparse_categorical_accuracy

Training_Period...: 100%|██████████| 1/1 [05:55<00:00, 355.86s/it]


In [20]:
test_true = test_data['label']
test_pred = np.argmax(tf.nn.softmax(model.predict(test_ds)), axis=-1)

45/45 [==============================] - 5s 102ms/step


In [21]:
from sklearn.metrics import classification_report
print(classification_report(test_true, test_pred))

              precision    recall  f1-score   support

           0       0.18      0.33      0.24      7692
           1       0.35      0.34      0.34     14802
           2       0.53      0.41      0.46     23272

    accuracy                           0.37     45766
   macro avg       0.35      0.36      0.35     45766
weighted avg       0.41      0.37      0.39     45766



In [22]:
model_pred_logit = model.predict(test_ds)
model_pred_prob = tf.nn.softmax(model_pred_logit)
model_pred_label = np.argmax(model_pred_prob, axis=-1)
model_pred_label_prob = np.max(model_pred_prob, axis=-1)

45/45 [==============================] - 2s 39ms/step


In [23]:
output_df = test_data[['stock_code', 'stock_name', 'datetime', 'label']]
output_df['label_pred'] = model_pred_label
output_df['label_prob'] = model_pred_label_prob
output_df.to_pickle(f'../../Offline/backtest/backtest_data/test/{benchmark}_stock_selection_results_{test_start_date}_cls.pkl') 

In [25]:
# output_df.head()